In [1]:
import os
from pathlib import Path
from openpyxl import Workbook, load_workbook
from typing import Union

def compile_workbooks(workbooks_path: Union[str, Path], final_filename: str) -> Path:
    """
    Compila múltiplas planilhas Excel em um único arquivo.
    
    Args:
        workbooks_path: Caminho para o diretório com as planilhas
        final_filename: Nome do arquivo final
    
    Returns:
        Path: Caminho do arquivo consolidado
    """
    workbooks_path = Path(workbooks_path)
    if not workbooks_path.is_dir():
        raise NotADirectoryError(f"'{workbooks_path}' não é um diretório válido.")
    
    if not final_filename.endswith('.xlsx'):
        final_filename += '.xlsx'
    
    final_wb = Workbook()
    final_ws = final_wb.active
    final_ws.title = "Consolidado"
    
    excel_files = list(workbooks_path.glob('*.xlsx'))
    if not excel_files:
        raise ValueError(f"Nenhum arquivo .xlsx encontrado em {workbooks_path}")
    
    #Processa o primeiro arquivo para obter o cabeçalho
    first_wb = load_workbook(excel_files[0], read_only=True)
    first_sheet = first_wb.active
    header = next(first_sheet.iter_rows(values_only=True))
    final_ws.append(header)
    
    #Processa todos os arquivos
    for file_path in excel_files:
        wb = load_workbook(file_path, read_only=True)
        for sheet in wb.worksheets:
            rows = sheet.iter_rows(min_row=2, values_only=True)
            for row in rows:
                if any(row):  #Ignora linhas vazias
                    final_ws.append(row)
        wb.close()
    
    #Salva o arquivo final - Arquivos_Unificados
    downloads_folder = Path.home() / "Downloads"
    output_path = downloads_folder / final_filename
    final_wb.save(output_path)
    
    return output_path

if __name__ == '__main__':
    try:
        downloads_folder = Path.home() / "Downloads"
        input_folder = downloads_folder / "workbooks_by_months"
        saved_path = compile_workbooks(input_folder, "Arquivos_Unificados")
        print(f"Arquivo consolidado salvo em: {saved_path}")
    except Exception as e:
        print(f"Erro: {e}")

Erro: 'C:\Users\educo\Downloads\workbooks_by_months' não é um diretório válido.
